<h1>all the code for Shm's Silly Shrek Demo</h1>

***stuff i did before class (to transcribe shrek.)***

you don't need to run this part, as shrek_timestamped.txt is already included ;)

1. Load up Shrek mp4...

In [ ]:
from ipywidgets import Output, GridspecLayout
from IPython.display import display
from IPython.display import Video
Video("shrek.mp4") #display the video! 

...and get its audio file as "audio.wav"

In [ ]:
video = AudioSegment.from_file("shrek.mp4", format="mp4")

audio = video.set_channels(1).set_frame_rate(16000).set_sample_width(2)
audio.export("audio.wav", format="wav")

2. Open up the audio file and use OpenAI's Whisper Model to recognize the speech and transcribe it as text. Then, print the result.

In [ ]:
# This code block may take a very long time to run! 
import whisper
from pydub import AudioSegment
from pathlib import Path
model = whisper.load_model("medium")
result = model.transcribe("audio.wav", language="English")
print(result)

In [ ]:
# Save our transcription as a text file
output_file = "shrek_timestamped.txt"
with open(output_file, "w") as file:
 # Writes the transcribed text into a file, with timestamps
    for segment in result["segments"]:
        start = segment["start"]
        end = segment["end"]
        text = segment["text"]
        file.write(f"[{start:.2f} - {end:.2f}] {text}\n")
print(f"Transcription saved to {output_file}")

In [ ]:
# the timestamps are in seconds; we might need them in seconds/hours/minutes.
# this is a helper function for that, later.
def seconds_to_timecode(s):
    seconds = float(s)
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = seconds % 60
    return f"{hours:02}:{minutes:02}:{seconds:06.3f}"

<h4>~ End of stuff I did before class ~</h4>


--------
<h1>Beginning of in-class demo!</h1>


*Let's open up our timestamped transcription of shrek!*


In [ ]:
shrek = open('shrek_timestamped.txt').read()

In [ ]:
print(shrek)

In [ ]:
print(shrek.upper())

In [ ]:
shrek = shrek.lower()
print(shrek)

<h1>donkey time!</h1>
Let's find all the donkeys!


In [ ]:
print(shrek.count('donkey'))

In [ ]:
donkeys = []
for index, line in enumerate(shrek.split('\n')):
    if 'donkey' in line.lower():
        segment, text = line.strip().split('] ', 1)
        start_time, end_time = segment[1:].split('-')
        entry = {'start': start_time, 'end': end_time, 'text': text}
        donkeys.append(entry)

In [ ]:
for donkey in donkeys:
    print(donkey)

In [ ]:
import ffmpeg

for i, donkey in enumerate(donkeys):
    start_time, end_time = donkey['start'], donkey['end']
    ffmpeg.input('shrek.mp4', ss=seconds_to_timecode(start_time), to=seconds_to_timecode(end_time)).output(f"clip_{i + 1}.mp4", codec='copy').run(overwrite_output=True, capture_stdout=True, capture_stderr=True)


In [ ]:
# Display all the video clips!
grid = GridspecLayout(1, len(donkeys))
for i, donkey in enumerate(donkeys[::2]):
    filepath = f"clip_{i + 1}.mp4"
    out = Output()
    with out:
        display.display(display.Video(filepath, embed=True))
    grid[0, i] = out
grid

**now let's deep fry all the videos**

In [ ]:
for i, donkey in enumerate(donkeys):
    num_clips = len(donkeys)
    start_time, end_time = donkey['start'], donkey['end']
    speed_factor =  (i/(num_clips)) * 15 + 1 
    
    
    # Visual deep frying
    contrast = ((i / (num_clips - 1)) * 10) + 1
    brightness = (i / num_clips * 0.1) + 0.1
    saturation = (i / num_clips * 2.0) + 1
    gamma_r = ((i / (num_clips - 1)) * 10) + 1 
    gamma_g = ((i / (num_clips - 1)) * 7) + 1
    gamma_b = ((i / (num_clips - 1)) * 5) + 1
    
    output_file = f"clip_{i + 1}_fried.mp4"
    input_stream = ffmpeg.input('shrek.mp4', ss=seconds_to_timecode(start_time), to=seconds_to_timecode(end_time))
    video = (
        input_stream.video
        .filter('setpts', f'PTS/{speed_factor}')
        .filter('eq', contrast=contrast, brightness=brightness, saturation=saturation, gamma_r=gamma_r, gamma_g=gamma_g, gamma_b=gamma_b, gamma_weight=0.2)
    )
    
    audio = input_stream.audio.filter('atempo', speed_factor)
    
    (
        ffmpeg
        .output(video, audio, output_file)
        .run(overwrite_output=True, quiet=True, capture_stdout=True, capture_stderr=True)
    )


In [ ]:
compiled = []

for i in range(1, len(donkeys) + 1):
    clip = ffmpeg.input(f"clip_{i}_fried.mp4")
    video_stream = clip.video
    audio_stream = clip.audio
    
    compiled.append(video_stream)
    compiled.append(audio_stream)

# Concatenate video and audio streams in pairs
merged = (
    ffmpeg
    .concat(*compiled, v=1, a=1)
    .output("fried_donkeys.mp4")
    .run(overwrite_output=True, capture_stdout=True, capture_stderr=True)
)


In [ ]:
Video("fried_donkeys.mp4", embed=True)

<h3>A lil extra demo that is more related to shm's research: Creativity Tools</h3>

1. let's make a simple drawing canvas... (a white rectangle that creates black dots when you click)

In [ ]:
from ipycanvas import Canvas
from ipywidgets import VBox
from IPython.display import display

canvas = Canvas(width=800, height=200)
canvas.fill_style = 'white'
canvas.fill_rect(0, 0, canvas.width, canvas.height)
canvas.stroke_style = 'black'
canvas.line_width = 3
canvas.fill_style = 'black'
canvas.stroke_rect(0, 0, canvas.width, canvas.height)

def on_mouse_down(x, y):
    canvas.mouse_down = True
    canvas.fill_circle(x, y, 12)
canvas.on_mouse_down(on_mouse_down)

display(VBox([canvas]))


2. let's make it so lots of dots appear continuously as you click *and drag*....

In [ ]:
# Create a canvas
canvas = Canvas(width=800, height=200)
canvas.fill_style = 'white'
canvas.fill_rect(0, 0, canvas.width, canvas.height)
canvas.stroke_style = 'black'
canvas.line_width = 3
canvas.fill_style = 'black'
canvas.stroke_rect(0, 0, canvas.width, canvas.height)
#initialize canvas variables
canvas.mouse_down = False
canvas.last_x = None
canvas.last_y = None

canvas.brush_spacing = 3 # default space between each "dot"

def lerp(a, b, t):
    return a + (b - a) * t

def on_mouse_down(x, y):
    canvas.mouse_down = True
    canvas.last_x, canvas.last_y = x, y
    canvas.fill_circle(x, y, 5)

def on_mouse_move(x, y):
    if canvas.mouse_down:
        dist = ((x - canvas.last_x)**2 + (y - canvas.last_y)**2) ** 0.5
        steps = int(dist / 3) # <-- Distance between the circles!!
        for i in range(steps):
            inter_x = lerp(canvas.last_x, x, i / steps)
            inter_y = lerp(canvas.last_y, y, i / steps)
            canvas.fill_circle(inter_x, inter_y, 5)
        canvas.fill_circle(x, y, 5)
        canvas.last_x, canvas.last_y = x, y

def on_mouse_up(x, y):
    canvas.mouse_down = False
    canvas.last_x, canvas.last_y = None, None

canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_move(on_mouse_move)
canvas.on_mouse_up(on_mouse_up)

display(VBox([canvas]))


3. let's add colors and a design a lil GUI for selecting them!

In [ ]:
from ipycanvas import Canvas
from ipywidgets import VBox

# Create a canvas
canvas = Canvas(width=800, height=200)
canvas.fill_style = 'white'
canvas.fill_rect(0, 0, canvas.width, canvas.height)
canvas.stroke_style = 'black'
canvas.line_width = 3
canvas.fill_style = 'black'

# Initialize canvas variables
canvas.mouse_down = False
canvas.last_x = None
canvas.last_y = None
canvas.brush_color = 'black'
canvas.brush_size = 4  # Default brush size

# Define color boxes (x, y, width, height, color)
canvas.color_boxes = [
    (10, 10, 30, 30, '#e76f51'),
    (50, 10, 30, 30, '#f4a261'),
    (90, 10, 30, 30, '#e9c46a'),
    (130, 10, 30, 30, '#2a9d8f'),
    (170, 10, 30, 30, 'black')
]

# Define brush size circles (x, y, radius)
canvas.brush_size_circles = [
    (220, 25, 4),  # Small circle
    (260, 25, 12)   # Big circle
]

def reset_ui():
    canvas.fill_style = "white"
    canvas.fill_rect(0, 0, 300, 60)
    canvas.stroke_style = "black"
    canvas.stroke_rect(0, 0, 300, 60)
    canvas.stroke_rect(0, 0, canvas.width, canvas.height)
    for x, y, width, height, color in canvas.color_boxes:
        canvas.fill_style = color
        canvas.fill_rect(x, y, width, height)

    # Reset brush color to the current selection
    canvas.fill_style = canvas.brush_color  
    
    # Draw brush size circles
    for x, y, radius in canvas.brush_size_circles:
        canvas.fill_circle(x, y, radius)


def lerp(a, b, t):
    return a + (b - a) * t

def on_mouse_down(x, y):
    # Check if the click is within any of the color boxes
    for box_x, box_y, box_width, box_height, color in canvas.color_boxes:
        if box_x <= x <= box_x + box_width and box_y <= y <= box_y + box_height:
            
            
            canvas.fill_style = color  # Change brush color
            canvas.brush_color = color  # Update brush color
            reset_ui()
            canvas.stroke_rect(box_x - 1, box_y - 1, box_width + 2, box_height + 2) #highlight selected box
            return
    
    # Check if the click is within any of the brush size circles
    for circle_x, circle_y, radius in canvas.brush_size_circles:
        if (x - circle_x)**2 + (y - circle_y)**2 <= radius**2:
            reset_ui()
            canvas.stroke_circle(circle_x, circle_y, radius + 2)  # Highlight the selected circle
            canvas.brush_size = radius  # Change brush size
            return
    
    # If not clicking a color box or brush size circle, start drawing
    canvas.mouse_down = True
    canvas.last_x, canvas.last_y = x, y
    canvas.fill_circle(x, y, canvas.brush_size)

def on_mouse_move(x, y):
    if canvas.mouse_down:
        dist = ((x - canvas.last_x)**2 + (y - canvas.last_y)**2) ** 0.5
        steps = int(dist / 2)
        for i in range(steps):
            inter_x = lerp(canvas.last_x, x, i / steps)
            inter_y = lerp(canvas.last_y, y, i / steps)
            canvas.fill_circle(inter_x, inter_y, canvas.brush_size)
        canvas.fill_circle(x, y, canvas.brush_size)
        canvas.last_x, canvas.last_y = x, y

def on_mouse_up(x, y):
    canvas.mouse_down = False
    canvas.last_x, canvas.last_y = None, None

# Bind the mouse events to the canvas
canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_move(on_mouse_move)
canvas.on_mouse_up(on_mouse_up)

# Display the canvas
display(VBox([canvas]))
reset_ui()
